On a platform of your choice, implement these three methods for computing the Fibonacci
numbers. For Parts (d)-(f) use integer variables. (You do not need to submit your source code with your assignment.)

d. (15 points)
How fast does
each method appear to be? (This is deliberately open-ended; part of the problem is to decide what constitutes a reasonable answer.)
Include precise timings if possible---you will need to figure out how to time processes on the system
you are using, if you do not already know.

e. (4 points)
What's the first Fibonacci number that's at least $2^{31}$? (If you're using C longs, this is where you hit
integer overflow.)

f. (15 points)
Since you should reach ``integer overflow'' with the faster methods quite quickly, modify your programs
so that they return the Fibonacci numbers modulo $2^{16}$. (In other words, make all of your
arithmetic modulo $2^{16}$---this will avoid overflow! You must do this regardless of whether or not your
system overflows.) For each method, what is the largest value of $k$ such that you can compute the
$k^\text{th}$ Fibonacci number (modulo 65536) in 5 seconds of machine time? If that value of $k$ would be too big to handle (e.g. if you'd get integer overflow on $k$ itself) but you can still calculate $F_{k}$ quickly, you may report the largest value $k_{\rm max}$ of $k$ you can handle and the amount of time the calculation of $F_{k_{\rm max}}$ takes. 

g. (4 points) Finally, Wikipedia also describes the formula     $F_N = \frac{\varphi^N - \psi^N}{\varphi-\psi}$ where $\varphi = \frac{1+\sqrt{5}}2$ and $\psi = 1-\varphi$. Due to floating point error, this computation will not always return an integer. Instead, we will round our result to the nearest integer, ie $F_N = \left\lceil \frac{\varphi^N - \psi^N}{\varphi-\psi} \right\rfloor$ where $\lceil x \rfloor$ is the result of $x$ being rounded to the nearest integer.


Using standard floating point arithmetic on your platform of choice, report the smallest $N$ for which the above formula does not correctly output the $N$th Fibonacci number.

In [1]:
import numpy as np
def fib_rec(n):
    if n == 0 or n == 1:
        return n
    return fib_rec(n-1) + fib_rec(n-2)

def fib_iter(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a+b
    return a

def mat_mult(A, B):
    return [[A[0][0]*B[0][0]+A[0][1]*B[1][0],
             A[0][0]*B[0][1]+A[0][1]*B[1][1]],
            [A[1][0]*B[0][0]+A[1][1]*B[1][0],
             A[1][0]*B[0][1]+A[1][1]*B[1][1]]]

def mat_pow(M, n):
    if n == 0:
        return [[1, 0], [0, 1]]
    if n == 1:
        return M
    half = mat_pow(M, n//2)
    sq = mat_mult(half, half)
    if n % 2 == 0:
        return sq
    else:
        return mat_mult(sq, M)

def fib_matrix(n):
    M = [[1, 1], [1, 0]]
    R = mat_pow(M, n)
    return R[1][0]

In [ ]:
import time
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def time_function(func, n):
    start = time.time()
    func(n)
    end = time.time()
    return end - start
    
rec_ns = [5,10,15,20,25,30,35, 40]
iter_ns = [5,10,15,20,25,30,35, 40, 1000,2000,4000,8000,16000, 32000]
mat_ns = [5,10,15,20,25,30,35, 40, 1000,2000,4000,8000,16000, 32000, 1000000,2000000,4000000,8000000,16000000,32000000]

rec_times = [time_function(fib_rec,n) for n in rec_ns]
iter_times = [time_function(fib_iter,n) for n in iter_ns]
mat_times = [time_function(fib_matrix,n) for n in mat_ns]

phi = (1+math.sqrt(5))/2

def exp_model(n,a):
    return a*(phi**n)

def quad_model(n,a):
    return a*(n**2)

def nlog_model(n,a):
    return a*n*np.log(n)

def karatsuba_model(n, a):
    return a*n**1.58

rec_params,_ = curve_fit(exp_model, rec_ns, rec_times)
iter_params,_ = curve_fit(quad_model, iter_ns, iter_times)
mat_params,_ = curve_fit(nlog_model, mat_ns, mat_times)
mat_params2,_ = curve_fit(quad_model, mat_ns, mat_times)
mat_params3,_ = curve_fit(karatsuba_model, mat_ns, mat_times)

def r_squared(y, yhat):
    ss_res = np.sum((y-yhat)**2)
    ss_tot = np.sum((y-np.mean(y))**2)
    return 1 - ss_res/ss_tot

rec_pred = exp_model(np.array(rec_ns), rec_params[0])
iter_pred = quad_model(np.array(iter_ns), iter_params[0])
mat_pred = nlog_model(np.array(mat_ns), mat_params[0])
mat_pred2 = quad_model(np.array(mat_ns), mat_params2[0])
mat_pred3 = karatsuba_model(np.array(mat_ns), mat_params3[0])

print("Recursive R^2:", r_squared(np.array(rec_times), rec_pred))
print("Iterative R^2:", r_squared(np.array(iter_times), iter_pred))
print("Matrix(nlogn) R^2:", r_squared(np.array(mat_times), mat_pred))
print("Matrix(n^2) R^2:", r_squared(np.array(mat_times), mat_pred2))
print("Matrix(n^1.58) R^2:", r_squared(np.array(mat_times), mat_pred3))

plt.figure()
plt.scatter(rec_ns, rec_times, label="Recursive data")
plt.plot(rec_ns, rec_pred, label="a*phi^n")
plt.xlabel("n")
plt.ylabel("Time (s)")
plt.title("Recursive Fibonacci")
plt.legend()
plt.show()

plt.figure()
plt.scatter(iter_ns, iter_times, label="Iterative data")
plt.plot(iter_ns, iter_pred, label="a*n^2")
plt.xlabel("n")
plt.ylabel("Time (s)")
plt.title("Iterative Fibonacci")
plt.legend()
plt.show()

plt.figure()
plt.scatter(mat_ns, mat_times, label="Matrix data")
plt.plot(mat_ns, mat_pred, label="a*n*log n")
plt.plot(mat_ns, mat_pred2, label="a*n^2")
plt.plot(mat_ns, mat_pred3, label="a*n^1.58 (Karatsuba)")
plt.xlabel("n")
plt.ylabel("Time (s)")
plt.title("Matrix Fibonacci")
plt.legend()
plt.show()

print("\nRUNTIME TABLE (seconds)\n")
print(f"{'Recursive n':>12} | {'Recursive':>12} | {'Iterative n':>12} | {'Iterative':>12} | {'Matrix n':>12} | {'Matrix':>12}")
print("-"*85)

max_len = max(len(rec_ns), len(iter_ns), len(mat_ns))
for i in range(max_len):
    n_rec = rec_ns[i] if i < len(rec_ns) else ""
    t_rec = f"{rec_times[i]:.6f}" if i < len(rec_times) else ""
    
    n_it = iter_ns[i] if i < len(iter_ns) else ""
    t_it = f"{iter_times[i]:.6f}" if i < len(iter_times) else ""
    
    n_mat = mat_ns[i] if i < len(mat_ns) else ""
    t_mat = f"{mat_times[i]:.6f}" if i < len(mat_times) else ""
    
    print(f"{str(n_rec):>12} | {t_rec:>12} | {str(n_it):>12} | {t_it:>12} | {str(n_mat):>12} | {t_mat:>12}")

In [6]:
MOD = 2**16

In [7]:
def fib_rec_mod(n):
    if n == 0 or n == 1:
        return n
    return (fib_rec_mod(n-1) + fib_rec_mod(n-2)) % MOD

In [8]:
def fib_iter_mod(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, (a + b) % MOD
    return a

In [9]:
def mat_mult_mod(A, B):
    return [[(A[0][0]*B[0][0] + A[0][1]*B[1][0]) % MOD,
             (A[0][0]*B[0][1] + A[0][1]*B[1][1]) % MOD],
            [(A[1][0]*B[0][0] + A[1][1]*B[1][0]) % MOD,
             (A[1][0]*B[0][1] + A[1][1]*B[1][1]) % MOD]]

def fib_matrix_mod(n):
    result = [[1,0],[0,1]]
    base = [[1,1],[1,0]]

    while n > 0:
        if n % 2 == 1:
            result = mat_mult_mod(result, base)
        base = mat_mult_mod(base, base)
        n //= 2

    return result[1][0]


In [ ]:
def max_k_in_time(f, start_k=1, limit=5.0):
    k = start_k
    while True:
        t0 = time.perf_counter()
        f(k)
        t1 = time.perf_counter()
        if t1 - t0 > limit:
            return k
        k *= 2

def max_k_in_time_recursive(f, limit=5.0):
    k = 5
    while k <= 40:
        t0 = time.perf_counter()
        f(k)
        t1 = time.perf_counter()
        if t1 - t0 > limit:
            return k
        k += 1
    return k

In [11]:
#print(time_function(fib_rec_mod, 37))
#print(time_function(fib_iter_mod, 82000000))
#print(time_function(fib_matrix_mod, 10**59200))
#print(time_function(fib_matrix_mod, 10**59200))
#print(time_function(fib_matrix_mod, 10**59200))
#print(time_function(fib_matrix_mod, 10**59200))
#print(time_function(fib_matrix_mod, 10**59200))
#print(time_function(fib_matrix_mod, 10**59200))
#print(time_function(fib_matrix_mod, 10**59200))
#print(time_function(fib_matrix_mod, 10**59200))
print(time_function(fib_matrix_mod, 10**59200))
#print(time_function(fib_matrix_mod, 2**200000))

5.011451244354248


In [ ]:
print("Recursive max k:", max_k_in_time_recursive(fib_rec_mod))
print("Iterative max k:", max_k_in_time(fib_iter_mod, 1000))
print("Matrix max k:", max_k_in_time(fib_matrix_mod, 100000))